In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import pandas as pd
import time


ModuleNotFoundError: No module named 'selenium'

In [ ]:
import requests
import pandas as pd
import time

all_records = []
page = 1
max_pages = 249  # znane z API

while page <= max_pages:
    url = f"https://cpj.org/wp-json/cpj-datamanager/v1/people_list?page={page}"
    print(f"Pobieram stronę {page}...")
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Błąd przy stronie {page}: {response.status_code}")
        break

    data = response.json()
    records = data.get("data", [])
    if not records:
        print("Brak danych – przerywam.")
        break

    all_records.extend(records)
    page += 1
    print(f"Obadana strona {page}")
    time.sleep(0.5)  # żeby nie obciążać serwera

# Tworzymy DataFrame z interesującymi nas kolumnami
df = pd.DataFrame(all_records)[["fullName", "organizations", "location", "status", "typeOfDeath", "startDisplay", "type", "motiveConfirmed", "charges"]]

# Zapis do pliku
df.to_csv("cpj_full_dataset.csv", index=False)
print(f"Pobrano {len(df)} rekordów i zapisano do cpj_full_dataset.csv")


Pobieram stronę 1...
Obadana strona 2
Pobieram stronę 2...
Obadana strona 3
Pobieram stronę 3...
Obadana strona 4
Pobieram stronę 4...
Obadana strona 5
Pobieram stronę 5...
Obadana strona 6
Pobieram stronę 6...
Obadana strona 7
Pobieram stronę 7...
Obadana strona 8
Pobieram stronę 8...
Obadana strona 9
Pobieram stronę 9...
Obadana strona 10
Pobieram stronę 10...
Obadana strona 11
Pobieram stronę 11...
Obadana strona 12
Pobieram stronę 12...
Obadana strona 13
Pobieram stronę 13...
Obadana strona 14
Pobieram stronę 14...
Obadana strona 15
Pobieram stronę 15...
Obadana strona 16
Pobieram stronę 16...
Obadana strona 17
Pobieram stronę 17...
Obadana strona 18
Pobieram stronę 18...
Obadana strona 19
Pobieram stronę 19...
Obadana strona 20
Pobieram stronę 20...
Obadana strona 21
Pobieram stronę 21...
Obadana strona 22
Pobieram stronę 22...
Obadana strona 23
Pobieram stronę 23...
Obadana strona 24
Pobieram stronę 24...
Obadana strona 25
Pobieram stronę 25...
Obadana strona 26
Pobieram stronę 2

In [ ]:
df

,fullName,organizations,location,status,typeOfDeath,startDisplay,type,motiveConfirmed,charges
0,Aamer Abdelmonem,Freelance,Egypt,Imprisoned,,"December 19, 2020",Journalist,,"Anti-state,False News"
1,Aasif Sultan,Kashmir Narrator,India,Imprisoned,,"August 27, 2018",Journalist,,Anti-state
2,Abadullah Hananzai,"Radio Azadi,Radio Free Europe/Radio Liberty",Afghanistan,Killed,Murder,"April 30, 2018",Journalist,Confirmed,None
3,Abay Hailu,Agiere,Ethiopia,Killed,Dangerous Assignment,"February 9, 1998",Journalist,Confirmed,None
4,Abay Hailu,"Agiere, Welafen",Ethiopia,Imprisoned,,"February 22, 1997",Journalist,,None
...,...,...,...,...,...,...,...,...,...
4975,Yelena Petrova,Lita-M,Russia,Missing,,"August 11, 1996",Journalist,,None
4976,Yeliz Kılıç,Yürüyüş,Turkey,Imprisoned,,"January 18, 2013",Journalist,,Anti-state
4977,Yemane Haile,Eritrean News Agency,Eritrea,Imprisoned,,"November 15, 2006",Journalist,,No Charge Disclosed
4978,Yensi Roberto Ordoñez Galdámez,Channel 14,Guatemala,Killed,Unknown,"May 19, 2011",Journalist,Unconfirmed,None


In [ ]:
# --- Konfiguracja Selenium (tryb headless) ---
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

# --- Pobieranie listy osób z JSON API ---
base_url = "https://cpj.org/wp-json/cpj-datamanager/v1/people_list"
people = []

print("⏳ Pobieram dane podstawowe z JSON...")

for page in range(0, 249):  # 249 stron
    print(f"📄 Strona {page}/249")
    resp = requests.get(base_url, params={"page": page})
    data = resp.json().get("data", [])
    people.extend(data)
    time.sleep(0.5)  # nie spamujemy serwera

# --- Scraping szczegółowych danych z mtpage ---
output = []

print("🔎 Rozpoczynam szczegółowy scraping...")

def get_elem_text_safe(by, value):
    try:
        elem = driver.find_element(by, value)
        return elem.text.strip()
    except:
        return ""

for idx, person in enumerate(people):
    try:
        url = person["mtpage"]
        driver.get(url)

        # Czekamy na załadowanie elementów z JS
        wait = WebDriverWait(driver, 10)
        jobs_elem = wait.until(EC.presence_of_element_located((By.ID, "jobs")))
        mediums_elem = driver.find_element(By.ID, "mediums")

        output.append({
            "fullName": person["fullName"],
            "location": person["location"],
            "organizations": person["organizations"],
            "status": person["status"],
            "typeOfDeath": person["typeOfDeath"],
            "startDisplay": person["startDisplay"],
            "type": person["type"],
            "motiveConfirmed": person["motiveConfirmed"],
            "charges": person["charges"],
            "gender": get_elem_text_safe(By.ID, "gender"),
            "localOrForeign": get_elem_text_safe(By.ID, "localOrForeign"),
            "sourcesOfFire": get_elem_text_safe(By.ID, "sourcesOfFire"),
            "coverages": get_elem_text_safe(By.ID, "coverages"),
            "jobs": get_elem_text_safe(By.ID, "jobs"),
            "mediums": get_elem_text_safe(By.ID, "mediums"),
            "sentence": get_elem_text_safe(By.ID, "sentence"),
            "impunity": get_elem_text_safe(By.ID, "impunity"),
            "tortured": get_elem_text_safe(By.ID, "tortured"),
            "threatened": get_elem_text_safe(By.ID, "threatened"),
            "captive": get_elem_text_safe(By.ID, "captive")
        })

        print(f"Sprawdzono {idx} osób") if (idx%50==0) else None

    except Exception as e:
        print(f"❌ Błąd przy {person['fullName']} — {str(e)}")
        output.append({
            "fullName": person["fullName"],
            "location": person["location"],
            "organizations": person["organizations"],
            "status": person["status"],
            "typeOfDeath": person["typeOfDeath"],
            "startDisplay": person["startDisplay"],
            "type": person["type"],
            "motiveConfirmed": person["motiveConfirmed"],
            "charges": person["charges"],
            "gender": get_elem_text_safe(By.ID, "gender"),
            "localOrForeign": get_elem_text_safe(By.ID, "localOrForeign"),
            "sourcesOfFire": get_elem_text_safe(By.ID, "sourcesOfFire"),
            "coverages": get_elem_text_safe(By.ID, "coverages"),
            "jobs": get_elem_text_safe(By.ID, "jobs"),
            "mediums": get_elem_text_safe(By.ID, "mediums"),
            "sentence": get_elem_text_safe(By.ID, "sentence"),
            "impunity": get_elem_text_safe(By.ID, "impunity"),
            "tortured": get_elem_text_safe(By.ID, "tortured"),
            "threatened": get_elem_text_safe(By.ID, "threatened"),
            "captive": get_elem_text_safe(By.ID, "captive")
        })

# --- Zamykamy przeglądarkę ---
driver.quit()

# --- Eksport danych ---
df = pd.DataFrame(output)
df.to_csv("cpj_full_dataset.csv", index=False)
print("💾 Zapisano do cpj_full_dataset.csv")


⏳ Pobieram dane podstawowe z JSON...
📄 Strona 0/249
📄 Strona 1/249
📄 Strona 2/249
📄 Strona 3/249
📄 Strona 4/249
📄 Strona 5/249
📄 Strona 6/249
📄 Strona 7/249
📄 Strona 8/249
📄 Strona 9/249
📄 Strona 10/249
📄 Strona 11/249
📄 Strona 12/249
📄 Strona 13/249
📄 Strona 14/249
📄 Strona 15/249
📄 Strona 16/249
📄 Strona 17/249
📄 Strona 18/249
📄 Strona 19/249
📄 Strona 20/249
📄 Strona 21/249
📄 Strona 22/249
📄 Strona 23/249
📄 Strona 24/249
📄 Strona 25/249
📄 Strona 26/249
📄 Strona 27/249
📄 Strona 28/249
📄 Strona 29/249
📄 Strona 30/249
📄 Strona 31/249
📄 Strona 32/249
📄 Strona 33/249
📄 Strona 34/249
📄 Strona 35/249
📄 Strona 36/249
📄 Strona 37/249
📄 Strona 38/249
📄 Strona 39/249
📄 Strona 40/249
📄 Strona 41/249
📄 Strona 42/249
📄 Strona 43/249
📄 Strona 44/249
📄 Strona 45/249
📄 Strona 46/249
📄 Strona 47/249
📄 Strona 48/249
📄 Strona 49/249
📄 Strona 50/249
📄 Strona 51/249
📄 Strona 52/249
📄 Strona 53/249
📄 Strona 54/249
📄 Strona 55/249
📄 Strona 56/249
📄 Strona 57/249
📄 Strona 58/249
📄 Strona 59/249
📄 Strona 60/2

In [ ]:
df.dtypes

fullName           object
location           object
organizations      object
status             object
typeOfDeath        object
startDisplay       object
type               object
motiveConfirmed    object
charges            object
gender             object
localOrForeign     object
sourcesOfFire      object
coverages          object
jobs               object
mediums            object
sentence           object
impunity           object
tortured           object
threatened         object
captive            object
dtype: object

In [1]:
#For Colab version
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
csv_path = '/content/drive/My Drive/Dzierz/cpj_full_dataset.csv'
df = pd.read_csv(csv_path)
df

Mounted at /content/drive


In [8]:
#Conda version
df = pd.read_csv('cpj_full_dataset.csv')
df

,fullName,location,organizations,status,typeOfDeath,startDisplay,type,motiveConfirmed,charges,gender,localOrForeign,sourcesOfFire,coverages,jobs,mediums,sentence,impunity,tortured,threatened,captive
0,Aamer Abdelmonem,Egypt,Freelance,Imprisoned,NaN,"December 19, 2020",Journalist,NaN,"Anti-state,False News",Male,Local,NaN,"Corruption, Human Rights, Politics","Columnist / Commentator, Editor, Internet Repo...","Internet, Print",Not sentenced,NaN,NaN,NaN,NaN
1,Aasif Sultan,India,Kashmir Narrator,Imprisoned,NaN,"August 27, 2018",Journalist,NaN,Anti-state,Male,Local,NaN,"Human Rights, Politics, War",Print Reporter,Print,Not sentenced,NaN,NaN,NaN,NaN
2,Abadullah Hananzai,Afghanistan,"Radio Azadi,Radio Free Europe/Radio Liberty",Killed,Murder,"April 30, 2018",Journalist,Confirmed,NaN,Male,Local,Political Group,"Crime, Politics, War",Producer,"Internet, Radio",NaN,Complete Impunity,No,No,No
3,Abay Hailu,Ethiopia,Agiere,Killed,Dangerous Assignment,"February 9, 1998",Journalist,Confirmed,NaN,NaN,NaN,NaN,"Human Rights, Politics",Print Reporter,Print,NaN,NaN,NaN,NaN,NaN
4,Abay Hailu,Ethiopia,"Agiere, Welafen",Imprisoned,NaN,"February 22, 1997",Journalist,NaN,NaN,Male,Local,Government Officials,"Human Rights, Politics",Print Reporter,Print,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,Yashar Agazadeh,Azerbaijan,Muhalifet,Imprisoned,NaN,"May 16, 2007",Journalist,NaN,Defamation,Male,Local,NaN,Politics,Print Reporter,Print,30 months,NaN,NaN,NaN,NaN
4976,Yasin al-Dulaimi,Iraq,Radio Al-Mustaqbal,Killed,Unknown,"December 26, 2006",Journalist,Unconfirmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4977,Yasir Kaya,Turkey,Fenerbahçe TV,Imprisoned,NaN,"August 12, 2017",Journalist,NaN,Anti-state,Male,Local,NaN,Sports,Broadcast Reporter,"Radio, Television",Not Sentenced,NaN,NaN,NaN,NaN
4978,Yasmina Brikh,Algeria,Algerian Radio,Killed,Murder,"September 4, 1995",Journalist,Confirmed,NaN,Female,Local,NaN,"Culture, Sports",Broadcast Reporter,Radio,NaN,Complete Impunity,No,No,No


In [9]:
df['location'].unique()

array(['Egypt', 'India', 'Afghanistan', 'Ethiopia', 'Iran', 'Syria',
       'Yemen', 'Kuwait', 'Libya',
       'Israel and the Occupied Palestinian Territory', 'Algeria',
       'Sudan', 'Lebanon', 'Iraq', 'Morocco', 'Somalia', 'Pakistan',
       'Ivory Coast', 'Niger', 'Bangladesh', 'South Africa',
       'Sierra Leone', 'China', 'Bahrain', 'Saudi Arabia', 'Turkey',
       'Maldives', 'Tunisia', 'Russia', 'Tajikistan', 'Guatemala',
       'Azerbaijan', 'Democratic Republic of the Congo', 'Cameroon',
       'South Sudan', 'USA', 'Mali', 'Argentina', 'Nigeria', 'Indonesia',
       'Peru', 'Mexico', 'Burundi', 'Rwanda', 'East Timor', 'Jordan',
       'Ghana', 'Eritrea', 'Singapore', 'Kazakhstan', 'Sri Lanka',
       'Gambia', 'Cuba', 'Philippines', 'Angola', 'Colombia',
       'Mozambique', 'Chad', 'Belarus', 'Central African Republic',
       'Georgia', 'Bosnia', 'Italy', 'Paraguay', 'Ukraine', 'El Salvador',
       'Comoros', 'Kyrgyzstan', 'Haiti', 'Nepal', 'Venezuela', 'Togo',
       

In [17]:
df['location'].replace({
    'Congo, Democratic Republic of the': 'Democratic Republic of the Congo',
    'USA': 'United States of America',
    'UK': 'United Kingdom',
    'Bosnia-Herzegovina': 'Bosnia and Herzegovina',
    'Ethopia': 'Ethiopia'
}, inplace=True)

In [18]:
df['location'].unique()

array(['Egypt', 'India', 'Afghanistan', 'Ethiopia', 'Iran', 'Syria',
       'Yemen', 'Kuwait', 'Libya',
       'Israel and the Occupied Palestinian Territory', 'Algeria',
       'Sudan', 'Lebanon', 'Iraq', 'Morocco', 'Somalia', 'Pakistan',
       'Ivory Coast', 'Niger', 'Bangladesh', 'South Africa',
       'Sierra Leone', 'China', 'Bahrain', 'Saudi Arabia', 'Turkey',
       'Maldives', 'Tunisia', 'Russia', 'Tajikistan', 'Guatemala',
       'Azerbaijan', 'Democratic Republic of the Congo', 'Cameroon',
       'South Sudan', 'United States of America', 'Mali', 'Argentina',
       'Nigeria', 'Indonesia', 'Peru', 'Mexico', 'Burundi', 'Rwanda',
       'East Timor', 'Jordan', 'Ghana', 'Eritrea', 'Singapore',
       'Kazakhstan', 'Sri Lanka', 'Gambia', 'Cuba', 'Philippines',
       'Angola', 'Colombia', 'Mozambique', 'Chad', 'Belarus',
       'Central African Republic', 'Georgia', 'Bosnia and Herzegovina',
       'Italy', 'Paraguay', 'Ukraine', 'El Salvador', 'Comoros',
       'Kyrgyzstan', 'H

In [ ]:
print(f'Wartości unikalne dla status: {df['status'].unique()}')
print(f'Wartości unikalne dla typeOfDeath: {df['typeOfDeath'].unique()}')
print(f'Wartości unikalne dla type: {df['type'].unique()}')
print(f'Wartości unikalne dla motiveConfirmed: {df['motiveConfirmed'].unique()}')
print(f'Wartości unikalne dla gender: {df['gender'].unique()}')
print(f'Wartości unikalne dla localOrForeign: {df['localOrForeign'].unique()}')
print(f'Wartości unikalne dla impunity: {df['impunity'].unique()}')
print(f'Wartości unikalne dla tortured: {df['tortured'].unique()}')
print(f'Wartości unikalne dla threatened: {df['threatened'].unique()}')
print(f'Wartości unikalne dla captive: {df['captive'].unique()}')

Wartości unikalne dla status: ['Imprisoned' 'Killed' nan 'Missing']
Wartości unikalne dla typeOfDeath: [nan 'Murder' 'Dangerous Assignment' 'Crossfire' 'Unknown'
 'Crossfire / Combat Related']
Wartości unikalne dla type: ['Journalist' 'Media Worker']
Wartości unikalne dla motiveConfirmed: [nan 'Confirmed' 'Unconfirmed']
Wartości unikalne dla gender: ['Male' nan 'Female']
Wartości unikalne dla localOrForeign: ['Local' nan 'Foreign']
Wartości unikalne dla impunity: [nan 'Complete Impunity' 'Partial Impunity' 'Full Justice']
Wartości unikalne dla tortured: [nan 'No' 'Yes']
Wartości unikalne dla threatened: [nan 'No' 'Yes']
Wartości unikalne dla captive: [nan 'No' 'Yes']


In [ ]:
print(f'Wartości unikalne dla mediums: {df['mediums'].unique()}')

Wartości unikalne dla mediums: ['Internet, Print' 'Print' 'Internet, Radio' 'Internet' nan
 'Internet, Print, Television' 'Internet, Television'
 'Documentary Film, Internet, Print' 'Television' 'Print, Television'
 'Radio' 'Radio, Television' 'Print, Radio'
 'Documentary Film, Internet, Television' 'Internet, Print, Radio'
 'Internet, Print, Radio, Television'
 'Documentary Film, Internet, Print, Radio, Television' 'Documentary Film'
 'Documentary Film, Internet' 'Print, Radio, Television'
 'Internet, Radio, Television' 'Documentary Film, Television' 'Unknown'
 'Documentary Film, Internet, Print, Television' 'Documentary Film, Print']


In [20]:
def clean_mediums(value):
    if pd.isna(value) or value == 'Unknown':
        return 'Unknown'
    mediums = [m.strip() for m in value.split(',')]
    return ', '.join(sorted(set(mediums)))

df['mediums_cleaned'] = df['mediums'].apply(clean_mediums)

In [21]:
print(df['mediums_cleaned'].unique())

['Internet, Print' 'Print' 'Internet, Radio' 'Internet' 'Unknown'
 'Internet, Print, Television' 'Internet, Television'
 'Documentary Film, Internet, Print' 'Television' 'Print, Television'
 'Radio' 'Radio, Television' 'Print, Radio'
 'Documentary Film, Internet, Television' 'Internet, Print, Radio'
 'Internet, Print, Radio, Television'
 'Documentary Film, Internet, Print, Radio, Television' 'Documentary Film'
 'Documentary Film, Internet' 'Print, Radio, Television'
 'Internet, Radio, Television' 'Documentary Film, Television'
 'Documentary Film, Internet, Print, Television' 'Documentary Film, Print']


In [ ]:
print(f'Wartości unikalne dla sourcesOfFire: {df['sourcesOfFire'].unique()}')

Wartości unikalne dla sourcesOfFire: [nan 'Political Group' 'Government Officials' 'Military Officials'
 'Government Officials, Military Officials' 'Criminal Group'
 'Paramilitary Group' 'Local Residents' 'Unknown Fire'
 'Criminal Group, Government Officials' 'Unknown'
 'Criminal Group, Local Residents, Military Officials, Paramilitary Group, Political Group'
 'Criminal Group, Political Group' 'Government Officials, Local Residents'
 'Mob Violence' 'Paramilitary Group, Political Group'
 'Criminal Group, Paramilitary Group'
 'Government Officials, Political Group' 'Criminal Group, Local Residents'
 'Criminal Group, Mob Violence' 'Criminal Group, Military Officials'
 'Military Officials, Paramilitary Group'
 'Local Residents, Political Group'
 'Criminal Group, Government Officials, Political Group'
 'Military Officials, Political Group'
 'Government Officials, Paramilitary Group'
 'Local Residents, Military Officials'
 'Government Officials, Mob Violence']


In [22]:
def clean_sources(value):
    if pd.isna(value):
        return 'Unknown'
    sources = [v.strip() for v in value.split(',')]
    return ', '.join(sorted(set(sources)))

df['sourcesOfFire_cleaned'] = df['sourcesOfFire'].apply(clean_sources)

In [23]:
print(df['sourcesOfFire_cleaned'].unique())

['Unknown' 'Political Group' 'Government Officials' 'Military Officials'
 'Government Officials, Military Officials' 'Criminal Group'
 'Paramilitary Group' 'Local Residents' 'Unknown Fire'
 'Criminal Group, Government Officials'
 'Criminal Group, Local Residents, Military Officials, Paramilitary Group, Political Group'
 'Criminal Group, Political Group' 'Government Officials, Local Residents'
 'Mob Violence' 'Paramilitary Group, Political Group'
 'Criminal Group, Paramilitary Group'
 'Government Officials, Political Group' 'Criminal Group, Local Residents'
 'Criminal Group, Mob Violence' 'Criminal Group, Military Officials'
 'Military Officials, Paramilitary Group'
 'Local Residents, Political Group'
 'Criminal Group, Government Officials, Political Group'
 'Military Officials, Political Group'
 'Government Officials, Paramilitary Group'
 'Local Residents, Military Officials'
 'Government Officials, Mob Violence']


In [ ]:
# for future
#df['sourcesOfFire'] = df['sourcesOfFire_cleaned']
#df.drop(columns=['sourcesOfFire_cleaned'], inplace=True)

In [ ]:
print(f'Wartości unikalne dla coverages: {df['coverages'].unique()}')

Wartości unikalne dla coverages: ['Corruption, Human Rights, Politics' 'Human Rights, Politics, War'
 'Crime, Politics, War' 'Human Rights, Politics' nan 'War' 'Politics'
 'Corruption, Human Rights, War' 'Human Rights, War'
 'Business, Corruption, Politics'
 'Business, Culture, Human Rights, Politics' 'Culture, Sports'
 'Corruption, Culture, Human Rights, Politics' 'Culture, Politics'
 'Human Rights' 'Corruption, Crime, Culture, Human Rights, Politics, War'
 'Crime'
 'Business, Corruption, Crime, Culture, Human Rights, Politics, Sports, War'
 'Corruption' 'Politics, War' 'Human Rights, Politics, Sports, War'
 'Culture, War' 'Unknown' 'Culture, Human Rights, War'
 'Corruption, Crime, Human Rights, Politics' 'Business, Crime, Culture'
 'Culture, Human Rights, Politics'
 'Business, Corruption, Crime, Culture, Human Rights, Politics, War'
 'Crime, Culture, Human Rights, Politics, War' 'Culture'
 'Corruption, Politics'
 'Business, Corruption, Crime, Culture, Human Rights, Politics, Sports'


In [ ]:
print(f'Wartości unikalne dla jobs: {df['jobs'].unique()}')

Wartości unikalne dla jobs: ['Columnist / Commentator, Editor, Internet Reporter, Print Reporter'
 'Print Reporter' 'Producer' 'Editor' 'Camera Operator'
 'Internet Reporter' nan 'Columnist / Commentator, Print Reporter'
 'Photographer' 'Internet Reporter, Print Reporter'
 'Columnist / Commentator' 'Camera Operator, Photographer'
 'Editor, Internet Reporter' 'Camera Operator, Photographer, Producer'
 'Internet Reporter, Publisher / Owner' 'Broadcast Reporter'
 'Broadcast Reporter, Columnist / Commentator, Internet Reporter'
 'Publisher / Owner' 'Broadcast Reporter, Editor, Producer'
 'Broadcast Reporter, Camera Operator' 'Print Reporter, Publisher / Owner'
 'Broadcast Reporter, Camera Operator, Photographer'
 'Columnist / Commentator, Internet Reporter'
 'Broadcast Reporter, Photographer, Publisher / Owner'
 'Camera Operator, Photographer, Publisher / Owner'
 'Internet Reporter, Photographer' 'Photographer, Producer'
 'Broadcast Reporter, Print Reporter' 'Unknown'
 'Columnist / Comment

In [ ]:
#df.loc[df['sentence'] == "1 year", 'sentence'] = "12"
df.loc[df['sentence'].isin(["1 year", "1 Year"]), 'sentence'] = "12"
df.loc[df['sentence'].isin(["2 years", "1 Years"]), 'sentence'] = "12"
df.loc[df['sentence'].isin(["3 years", "1 Years"]), 'sentence'] = "12"
df.loc[df['sentence'].isin(["4 years", "1 Years"]), 'sentence'] = "12"
df.loc[df['sentence'].isin(["5 years", "1 Years"]), 'sentence'] = "12"
df.loc[df['sentence'].isin(["6 years", "1 Years"]), 'sentence'] = "12"
df.loc[df['sentence'].isin(["7 years", "1 Years"]), 'sentence'] = "12"
df.loc[df['sentence'].isin(["8 years", "1 Years"]), 'sentence'] = "12"
df.loc[df['sentence'].isin(["9 years", "1 Years"]), 'sentence'] = "12"
df.loc[df['sentence'].isin(["10 years", "1 Years"]), 'sentence'] = "12"


In [ ]:
print(f'Wartości unikalne dla sentence: {df['sentence'].unique()}')

Wartości unikalne dla sentence: ['Not sentenced' nan '12' 'Not Sentenced' 'Life' '1 year 6 months'
 '4 years 2 months' '6 years 3 months' '13 months' '19 years'
 'not sentenced' '18 months' '12 years' '6-8 Years' '6 months' '15 years'
 '7 years 6 months' '2 Years 6 Months' '30 months' '36 months' 'Acquitted'
 '10 years 6 months' '10 months' '8 years 9 months' '36 years' '14 years'
 '8 years 6 months' '3 years 6 months' '14 months' '4 years 6 months'
 '2 years 6 months' '26 years' 'Unknown' '19 years, 6 months' '15 days'
 '32 years' '2 years 3 months' '4 years 4 months' '4 months'
 '5 years 6 months' '2 months' '3 years 9 months'
 '3 years 1 month 15 days' '17 years' '20 years' '12 years 6 months'
 '6 month suspended sentence' '8 months' '30 years' '5 years 10 months'
 '16 years' '1 year 3 months' '3 months' '1 year 9 months' '13 years'
 '4 years 5 months' '16 years 10 months' '15 months' '3 years 4 months'
 '4 years and 6 months' '6 years 6 months' '12 Years' '13 years 1 month'
 '3 Yea

In [ ]:
print(f'Wartości unikalne dla charges: {df['charges'].unique()}')

Wartości unikalne dla charges: ['Anti-state,False News' 'Anti-state' nan 'No Charge Disclosed'
 'Defamation' 'Anti-state,False News,Retaliatory'
 'Anti-state,Ethnic Or Religious Insult,Retaliatory' 'Retaliatory'
 'Anti-state,Retaliatory' 'False News,Retaliatory'
 'Ethnic Or Religious Insult' 'Defamation,Retaliatory'
 'Defamation,False News' 'False News' 'Censorship Violation'
 'Anti-state,Defamation,False News'
 'Anti-state,Defamation,Ethnic Or Religious Insult'
 'Anti-state,Defamation,Retaliatory'
 'Anti-state,Ethnic Or Religious Insult,False News'
 'Anti-state,Defamation' 'Anti-state,Censorship Violation'
 'Anti-state,Ethnic Or Religious Insult'
 'Defamation,False News,Retaliatory' 'Censorship Violation,Retaliatory'
 'No Charge Disclosed,Unknown' 'Unknown'
 'Defamation,Ethnic Or Religious Insult,False News,Retaliatory'
 'Defamation,Ethnic Or Religious Insult'
 'Censorship Violation,Defamation,False News'
 'Anti-state,Defamation,False News,Retaliatory']
